In [3]:
from langchain_core.tools import tool, StructuredTool
from typing import Optional


@tool
def multiply(a: Optional[int], b: int) -> int:
    """Multiply two numbers."""
    return a * b


# Let's inspect some of the attributes associated with the tool.
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'title': 'A'}, 'b': {'title': 'B', 'type': 'integer'}}


---

In [5]:
"""Base class for Google Calendar tools."""

from __future__ import annotations

from typing import TYPE_CHECKING

from langchain_core.tools import BaseTool
from pydantic import Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass

class GoogleCalendarBaseTool(BaseTool):
    api_resource: Resource = Field(default_factory=build_resource_service)
    
    @classmethod
    def from_api_resource(cls, api_resource: Resource) -> "GoogleCalendarBaseTool":
        """Create a tool from an api resource.

        Args:
            api_resource: The api resource to use.

        Returns:
            A tool.
        """
        return cls(api_resource=api_resource)

In [56]:
"""Create an event in Google Calendar."""

from datetime import datetime
from typing import Any, Dict, Optional, Type, Union

from langchain_core.callbacks import CallbackManagerForToolRun

from pydantic import BaseModel, Field


def get_current_datetime() -> str:
        """Get the current datetime."""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class CreateEventSchema(BaseModel):
    """Input for CreateEventTool."""

    summary: str = Field(
        ...,
        description="The title of the event.",
    )
    start_datetime: str = Field(
        default=get_current_datetime(),
        description=("The start datetime for the event in 'YYYY-MM-DD HH:MM:SS' format"
                     "The current year is 2024")
                    #  "if not provided in full, or expressions such as 'today', 'tomorrow', 'this afternoon', etc. are used."
                    #  "it must be set to 'NOW'."),
    )
    end_datetime: str = Field(
        ...,
        description="The end datetime for the event in 'YYYY-MM-DD HH:MM:SS' format",
    )
    location: Optional[str] = Field(
        default=None,
        description="The location of the event."
    )
    description: Optional[str] = Field(
        default=None,
        description="The description of the event."
    )


class CalendarCreateEvent(GoogleCalendarBaseTool):
    """Tool that create a event in Google Calendar."""

    name: str = "create_calendar_event"
    description: str = (
        "Use this tool to create an event." 
        "The input must be the summary, start and end datetime for the event."
    )
    args_schema: Type[CreateEventSchema] = CreateEventSchema

    def __get_timeZone(self) -> str:
        """Get the timezone of the primary calendar."""
        calendars = self.api_resource.calendarList().list().execute()
        return calendars['items'][0]['timeZone']

    def _prepare_event(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        location: Optional[str] = None,
        description: Optional[str] = None
    ) -> Dict[str, Any]:
        """Prepare the event body."""
        try: 
            date_object = datetime.strptime(start_datetime, "%Y-%m-%d %H:%M:%S")
            start = date_object.astimezone().replace(microsecond=0).isoformat()
            date_object = datetime.strptime(end_datetime, "%Y-%m-%d %H:%M:%S")
            end = date_object.astimezone().replace(microsecond=0).isoformat()
        except ValueError:
            raise ValueError("The datetime format is incorrect. Please use 'YYYY-MM-DD HH:MM:SS' format.")

        timezone = self.__get_timeZone()
        
        event = {
            "summary": summary,
            "location": location,
            "description": description,
            "start": {"dateTime": start, "timeZone": timezone},
            "end": {"dateTime": end, "timeZone": timezone},
        }
        return event
    
    def _run(
        self,
        summary: str,
        start_datetime: str,
        end_datetime: str,
        location: Optional[str] = None,
        description: Optional[str] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Run the tool."""
        try:
            body = self._prepare_event(summary, start_datetime, end_datetime, location=location, description=description)
            event = self.api_resource.events().insert(calendarId='primary', body=body).execute()
            return event.get('htmlLink')
        except Exception as error:
            raise Exception(f"An error occurred: {error}")


In [57]:
from __future__ import annotations

from typing import TYPE_CHECKING, List

from langchain_community.agent_toolkits.base import BaseToolkit
from langchain_core.tools import BaseTool
from pydantic import ConfigDict, Field

from langchain_google_community.gmail.utils import build_resource_service

if TYPE_CHECKING:
    # This is for linting and IDE typehints
    from googleapiclient.discovery import Resource  # type: ignore[import]
else:
    try:
        # We do this so pydantic can resolve the types when instantiating
        from googleapiclient.discovery import Resource
    except ImportError:
        pass


SCOPES = ["https://www.googleapis.com/auth/calendar"]

class GoogleCalendarToolkit(BaseToolkit):
    """Toolkit for interacting with GoogleCalendar."""

    api_resource: Resource = Field(default_factory=build_resource_service)

    model_config = ConfigDict(
        arbitrary_types_allowed=True,
    )

    def get_tools(self) -> List[BaseTool]:
        """Get the tools in the toolkit."""
        return [
            CalendarCreateEvent(api_resource=self.api_resource),
        ]

In [58]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)


# Can review scopes here https://developers.google.com/gmail/api/auth/scopes
# For instance, readonly scope is 'https://www.googleapis.com/auth/gmail.readonly'
credentials = get_gmail_credentials(
    token_file="token.json",
    scopes=["https://www.googleapis.com/auth/calendar"],
    client_secrets_file="credentials.json",
)
api_resource = build_resource_service(credentials=credentials, service_name='calendar', service_version='v3')
toolkit = GoogleCalendarToolkit(api_resource=api_resource)

In [59]:
tools = toolkit.get_tools()
tools

[CalendarCreateEvent(api_resource=<googleapiclient.discovery.Resource object at 0x11ac65160>)]

In [60]:
import os
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model=os.getenv('OPENAI_MODEL'), api_key=os.getenv('OPENAI_API_KEY'))

In [61]:
model_with_tools = model.bind_tools(tools)

In [62]:
from langchain_core.messages import HumanMessage

prompt = "Create a event in Google Calendar para el 10 de diciembre a las 10:00am con el titulo 'Reunion con el equipo'."

response = model_with_tools.invoke([HumanMessage(content=prompt)])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'create_calendar_event', 'args': {'summary': 'Reunion con el equipo', 'start_datetime': '2024-12-10 10:00:00', 'end_datetime': '2024-12-10 11:00:00'}, 'id': 'call_BC3xS9ULjfzHwqj6sdiUc5Os', 'type': 'tool_call'}]


---

In [63]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [69]:
example_query = "Create an event with title 'BETO y Alondra' in july 11, 2026 at 10:00am"

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Create an event with title 'BETO y Alondra' in july 11, 2026 at 10:00am
================================== Ai Message ==================================
Tool Calls:
  create_calendar_event (call_P392N1KfMqZ22OpEjSBsNqml)
 Call ID: call_P392N1KfMqZ22OpEjSBsNqml
  Args:
    summary: BETO y Alondra
    start_datetime: 2026-07-11 10:00:00
    end_datetime: 2026-07-11 11:00:00
================================= Tool Message =================================
Name: create_calendar_event

https://www.google.com/calendar/event?eid=MDBlZDdxcThzbHNoYjFqMG1wOWsyNGI2M3Mgam9yZ2VhbmczM0Bt
================================== Ai Message ==================================

The event titled "BETO y Alondra" has been created for July 11, 2026, at 10:00 AM. You can view it [here](https://www.google.com/calendar/event?eid=MDBlZDdxcThzbHNoYjFqMG1wOWsyNGI2M3Mgam9yZ2FubzM0Bt).
